# Task: Next word Prediction

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


**Load the dataset**

In [ ]:
with open('/sherlock_holmes.txt', 'r', encoding='utf-8') as file:
    text_data = file.read()


**Tokenize the words using space as a separator**

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text_data])
total_words = len(tokenizer.word_index) + 1


**Convert text data to sequences of integers**

In [ ]:
input_sequences = []
for line in text_data.split('\n'):
    token_list = line.split()  # Split words by spaces
    token_list = tokenizer.texts_to_sequences([token_list])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


**Pad sequences to have the same length**

In [ ]:
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')


**Separate input and output data**

In [ ]:
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

### **Build and compile the RNN model**

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(256))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
2569/2569 [==============================] - 42s 13ms/step - loss: 6.0391 - accuracy: 0.0919
Epoch 2/100
2569/2569 [==============================] - 19s 8ms/step - loss: 5.3032 - accuracy: 0.1429
Epoch 3/100
2569/2569 [==============================] - 18s 7ms/step - loss: 4.9217 - accuracy: 0.1671
Epoch 4/100
2569/2569 [==============================] - 18s 7ms/step - loss: 4.5714 - accuracy: 0.1838
Epoch 5/100
2569/2569 [==============================] - 19s 7ms/step - loss: 4.2097 - accuracy: 0.2040
Epoch 6/100
2569/2569 [==============================] - 18s 7ms/step - loss: 3.8375 - accuracy: 0.2354
Epoch 7/100
2569/2569 [==============================] - 19s 7ms/step - loss: 3.4759 - accuracy: 0.2801
Epoch 8/100
2569/2569 [==============================] - 18s 7ms/step - loss: 3.1402 - accuracy: 0.3314
Epoch 9/100
2569/2569 [==============================] - 18s 7ms/step - loss: 2.8389 - accuracy: 0.3834
Epoch 10/100
2569/2569 [==============================] - 18s 7

### **Train the RNN model**

### **Generate predictions**

**Function to generate the next word given a seed word**

In [ ]:
def generate_next_word(seed_text, next_words):
    for _ in range(next_words):
        token_list = seed_text.split()
        token_list = tokenizer.texts_to_sequences([token_list])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted_index = np.argmax(model.predict(token_list), axis=-1)
        predicted_word = tokenizer.index_word[predicted_index[0]]
        seed_text += " " + predicted_word
    return seed_text

**Provide a seed text to generate the next word**

In [ ]:
seed_text = "Sherlock Holmes"
next_words = 10
generated_text = generate_next_word(seed_text, next_words)
print("Generated text:", generated_text)

1/1 [==============================] - 0s 18ms/step
Generated text: Sherlock Holmes sat up with a “by said said he his precaution
